# Optimizing EV Charging Network using Chinese Remainder Theorem

### Step 1: Gather data on EV charging stations
Use an API like Open Charge Map (https://openchargemap.org/site/develop/api) to fetch the location of EV charging
stations in the area of interest. Make sure to obtain the latitude and longitude coordinates, as well as any relevant
metadata (e.g., charging speed, connector types).

In [1]:
! pip install folium requests geopy

     -------------------------------------- 102.3/102.3 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 125.4/125.4 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [2]:
import requests
import numpy as np
import folium

def fetch_charging_stations(api_key, country_code, max_results=100):
    url = "https://api.openchargemap.io/v3/poi/"
    params = {
        "key": api_key,
        "countrycode": country_code,
        "output": "json",
        "maxresults": max_results
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        charging_stations = response.json()
        if len(charging_stations) > 0:
            return charging_stations
        else:
            raise Exception("No charging stations found in the area of interest")
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

api_key = "YOUR_API_KEY"
country_code = "US"
charging_stations = fetch_charging_stations(api_key, country_code)

### Step 2: Gather data on electric vehicle models
Collect information on various EV models, including their range and charging capabilities. This data can be obtained
from manufacturer websites or other reliable sources.

In [3]:
# STEP 2
ev_models = [
    {"name": "Tesla Model 3", "range": 358, "charging_speed": 250},
    {"name": "Nissan Leaf", "range": 226, "charging_speed": 100},
    # Add more EV models here
]

### Step 3: Define the Chinese Remainder Theorem function
Create a function that takes the distance between two charging stations and the range of an EV model as input and
returns the remainder when the distance is divided by the range. This will help determine the optimal spacing between
charging stations.

In [4]:
# STEP 3

def modular_remainder(a, b):
    return a % b

def chinese_remainder(distances, ev_models):
    remainders = {}
    for model in ev_models:
        model_name = model["name"]
        remainders[model_name] = []
        for _, _, distance in distances:
            remainder = modular_remainder(distance, model["range"])
            remainders[model_name].append(remainder)
    # Find the minimum distance that satisfies the range of each EV model
    min_distance = {}
    for model in ev_models:
        model_name = model["name"]
        min_distance[model_name] = None
        for i, remainder in enumerate(remainders[model_name]):
            if remainder <= model["range"] - distances[i][2]:
                if min_distance[model_name] is None or distances[i][2] < min_distance[model_name]:
                    min_distance[model_name] = distances[i][2]
    return min_distance

### Step 4: Calculate distances between charging stations
Using the latitude and longitude coordinates obtained in Step 1, calculate the distance between each pair of charging
stations. You can use the Haversine formula or a library like geopy (https://geopy.readthedocs.io/en/stable/) to
compute the distances.

In [5]:
# STEP 4
from geopy.distance import distance

def calculate_distances(charging_stations):
    coords = [(station["AddressInfo"]["Latitude"], station["AddressInfo"]["Longitude"]) for station in charging_stations]
    distances = []
    for i in range(len(charging_stations)):
        for j in range(i+1, len(charging_stations)):
            dist = distance(coords[i], coords[j]).km
            distances.append((charging_stations[i]["ID"], charging_stations[j]["ID"], dist))
    return distances

distances = calculate_distances(charging_stations)

### Step 5: Analyze the distribution of charging stations
For each EV model, apply the modular arithmetic function to the distances between charging stations. Identify any patterns or clusters in the remainders, which may indicate inefficient spacing.

In [6]:
# Step 5
def analyze_remainders(distances, ev_models):
    remainders = {}
    for model in ev_models:
        model_name = model["name"]
        remainders[model_name] = []
        for _, _, distance in distances:
            remainder = modular_remainder(distance, model["range"])
            remainders[model_name].append(remainder)
    return remainders

remainders = analyze_remainders(distances, ev_models)

### Step 6: Optimize charging station placement
Develop an algorithm that iteratively adjusts the placement of charging stations to minimize the sum of the remainders from the modular arithmetic function. This can be achieved using optimization techniques like gradient descent or genetic algorithms.

In [7]:
import numpy as np
from scipy.optimize import linprog
import folium

def chinese_remainder_crt(remainders, moduli):
    M = np.prod(moduli)
    x = 0
    for i, (r, m) in enumerate(zip(remainders, moduli)):
        Mi = M // m
        if np.linalg.det(np.atleast_2d(Mi)) == 0:
            continue
        Mi_inv = int(np.linalg.matrix_power(Mi.reshape(1, -1), -1) % m)
        x += r * Mi * Mi_inv
    return x % M

def modular_remainder(a, m):
    return a % m

def optimize_charging_stations(distances, ev_models):
    optimized_distances = distances.copy()
    for model in ev_models:
        model_name = model["name"]
        remainders = [station[2] % model["range"] for station in distances]
        lcm = np.lcm.reduce([model["range"] for _ in range(len(distances))])
        crt_remainders = chinese_remainder_crt(remainders, [model["range"] for _ in range(len(distances))])

        # Formulate the problem as a linear programming problem
        c = np.ones(len(optimized_distances))
        A = np.zeros((lcm, len(optimized_distances)))
        b = np.zeros(lcm)

        for i, (_, _, distance) in enumerate(optimized_distances):
            for j in range(lcm):
                remainder = modular_remainder(distance, model["range"])
                A[j][i] = 1 if remainder == crt_remainders and crt_remainders != 0 else 0
                distance += 0.1

        bounds = [(0, None) for _ in range(len(optimized_distances))]

        # Solve the linear programming problem using the simplex method
        result = linprog(c, A_ub=A, b_ub=b, bounds=bounds)
        adjusted_distances = result.x

        # Update the optimized distances
        for i in range(len(optimized_distances)):
            optimized_distances[i] = (optimized_distances[i][0], optimized_distances[i][1], adjusted_distances[i])

    return optimized_distances

In [8]:
# import numpy as np
# from scipy.optimize import linprog
# import folium
# from itertools import permutations
# import numpy as np
# from scipy.optimize import linprog
# import folium

# def chinese_remainder_crt(remainders, moduli):
#     M = np.prod(moduli)
#     x = 0
#     for i, (r, m) in enumerate(zip(remainders, moduli)):
#         Mi = M // m
#         if np.linalg.det(np.atleast_2d(Mi)) == 0:
#             continue
#         Mi_inv = int(np.linalg.matrix_power(Mi.reshape(1, -1), -1) % m)
#         x += r * Mi * Mi_inv
#     return x % M


# def calculate_distance(coord1, coord2):
#     # Function to calculate the distance between two coordinates
#     # Implement your own distance calculation logic here
#     return ...

# def optimize_charging_stations(distances, ev_models):
#     optimized_distances = distances.copy()
#     for model in ev_models:
#         model_name = model["name"]
#         remainders = [station[2] % model["range"] for station in distances]
#         crt_remainders = chinese_remainder_crt(remainders, [model["range"] for _ in range(len(distances))])

#         # Generate all permutations of charging stations
#         permutations_list = list(permutations(range(len(optimized_distances))))

#         shortest_distance = float('inf')
#         best_permutation = None

#         # Iterate over all permutations to find the shortest route
#         for permutation in permutations_list:
#             total_distance = 0
#             prev_station = permutation[-1]
#             for station in permutation:
#                 total_distance += calculate_distance(optimized_distances[prev_station][:2], optimized_distances[station][:2])
#                 prev_station = station

#             if total_distance < shortest_distance:
#                 shortest_distance = total_distance
#                 best_permutation = permutation

#         # Update the optimized distances based on the best permutation
#         adjusted_distances = [optimized_distances[i] for i in best_permutation]

#         for i in range(len(optimized_distances)):
#             optimized_distances[i] = adjusted_distances[i]

#     return optimized_distances

### Step 7: Visualize the results
Create a map visualization of the optimized charging station locations using a library like Folium (https://python-visualization.github.io/folium/). Display the original and optimized locations, as well as the routes between them, to demonstrate the improvements in accessibility and efficiency.

In [9]:
def create_charging_stations_map(charging_stations, optimized_distances):
    m = folium.Map(location=[charging_stations[0]["AddressInfo"]["Latitude"], charging_stations[0]["AddressInfo"]["Longitude"]], zoom_start=10)

    # create markers for each charging station
    [folium.Marker((station["AddressInfo"]["Latitude"], station["AddressInfo"]["Longitude"])).add_to(m) for station in charging_stations]

    # create polylines between optimized charging stations
    for id1, id2, distance in optimized_distances:
        station1 = next(station for station in charging_stations if station["ID"] == id1)
        station2 = next(station for station in charging_stations if station["ID"] == id2)
        coord1 = (station1["AddressInfo"]["Latitude"], station1["AddressInfo"]["Longitude"])
        coord2 = (station2["AddressInfo"]["Latitude"], station2["AddressInfo"]["Longitude"])
        folium.PolyLine([coord1, coord2], color="blue", weight=2.5, opacity=0.7).add_to(m)
        return m
charging_stations = fetch_charging_stations(api_key, country_code)
distances = calculate_distances(charging_stations)
remainders = analyze_remainders(distances, ev_models)
optimized_distances = optimize_charging_stations(distances, ev_models)
create_charging_stations_map(charging_stations, optimized_distances)

C:\Users\KARTIKYA\AppData\Local\Temp\ipykernel_28496\2840580950.py:14: RuntimeWarning: divide by zero encountered in scalar remainder
  return x % M


### Step 8: Evaluate the performance of the optimization algorithm
Compare the original and optimized charging station distributions by calculating metrics like average waiting time, total distance traveled, and accessibility. This will help quantify the improvements achieved through the optimization process.

In [10]:
# STEP 8
def evaluate_performance(remainders, optimized_distances, ev_models):
    original_remainders_sum = {model["name"]: sum(remainders[model["name"]]) for model in ev_models}
    optimized_remainders = analyze_remainders(optimized_distances, ev_models)
    optimized_remainders_sum = {model["name"]: sum(optimized_remainders[model["name"]]) for model in ev_models}

    performance = {
        "original": original_remainders_sum,
        "optimized": optimized_remainders_sum
    }
    return performance

remainders = analyze_remainders(distances, ev_models)
optimized_distances = optimize_charging_stations(distances, ev_models)
performance = evaluate_performance(remainders, optimized_distances, ev_models)
print(performance)

C:\Users\KARTIKYA\AppData\Local\Temp\ipykernel_28496\2840580950.py:14: RuntimeWarning: divide by zero encountered in scalar remainder
  return x % M


{'original': {'Tesla Model 3': 463348.184505201, 'Nissan Leaf': 437444.1845052007}, 'optimized': {'Tesla Model 3': 0.0, 'Nissan Leaf': 0.0}}


### Step 9: Implement user input and customization
Allow users to input their own EV model information, as well as preferences for charging station density and other factors. Update the optimization algorithm to incorporate this user input and generate personalized recommendations for charging station placement.

In [11]:
# STEP 9
def get_user_input():
    name = input("Enter the name of your EV model: ")
    ev_range = float(input("Enter the range of your EV model (in miles): "))
    while ev_range < 0:
        print("Please enter a positive number.")
        ev_range = float(input("Enter the range of your EV model (in miles): "))
    charging_speed = float(input("Enter the charging speed of your EV model (in kW): "))
    while charging_speed < 0:
        print("Please enter a positive number.")
        charging_speed = float(input("Enter the charging speed of your EV model (in kW): "))
    density_preference = float(input("Enter your preferred charging station density (stations per mile): "))

    user_ev_model = {
        "name": name,
        "range": ev_range,
        "charging_speed": charging_speed,
        "density_preference": density_preference
    }
    return user_ev_model

user_ev_model = get_user_input()

### Step 10: Test and refine the algorithm
Test the algorithm on various datasets and scenarios to ensure its robustness and accuracy. Refine the algorithm as needed based on the results of these tests, and continue to iterate on the optimization process to improve its performance.

In [12]:
# Step 10
test_charging_stations = [
    {
        "ID": 1,
        "AddressInfo": {
            "Latitude": 37.774929,
            "Longitude": -122.419416
        }
    },
    {
        "ID": 2,
        "AddressInfo": {
            "Latitude": 37.775206,
            "Longitude": -122.417949
        }
    },
    {
        "ID": 3,
        "AddressInfo": {
            "Latitude": 37.775363,
            "Longitude": -122.414771
        }
    },
    {
        "ID": 4,
        "AddressInfo": {
            "Latitude": 37.773592,
            "Longitude": -122.414372
        }
    },
    {
        "ID": 5,
        "AddressInfo": {
            "Latitude": 37.772958,
            "Longitude": -122.418411
        }
    }
]

test_ev_models = [
    {
        "name": "Model X",
        "range": 300,
        "charging_speed": 120,
        "density_preference": 0.2
    },
    {
        "name": "Model Y",
        "range": 250,
        "charging_speed": 90,
        "density_preference": 0.25
    }
]

def optimize_charging_stations(distances, ev_models):
    # Initialize variables
    charging_stations = [i for i in range(len(distances))]
    remainders = {model["name"]: [model["range"]] * len(distances) for model in ev_models}
    optimized_stations = []
    optimized_distances = []

    # Sort charging stations by proximity to origin
    charging_stations = sorted(charging_stations, key=lambda x: distances[0][x])

    # Iterate over charging stations
    for station in charging_stations:
        # Calculate distances to all other charging stations
        distances_to_others = [(station, other, distances[station][other]) for other in charging_stations if other != station]
        distances_to_others = sorted(distances_to_others, key=lambda x: x[2])

        # Check if any EV model can reach this station from a previous station
        for model in ev_models:
            for idx, remainder in enumerate(remainders[model["name"]]):
                if remainder >= distances[optimized_stations[-1]][station] and distances[0][station] <= model["density_preference"]**(-1):
                    remainders[model["name"]][idx] = max(0, remainder - distances[optimized_stations[-1]][station])
                    optimized_stations.append(station)
                    break

        # Check if all EV models can reach this station from a previous station
        while len(optimized_stations) > 0:
            can_reach = True
            for model in ev_models:
                if remainders[model["name"]][0] < distances[optimized_stations[-1]][station]:
                    can_reach = False
                    break
            if can_reach:
                break
            else:
                optimized_stations.pop(-1)

    # If no valid charging stations are found within the specified range, reduce the range by 10% and try again
    range_miles = 10
    nearby_stations = find_nearby_charging_stations(user_location, charging_stations, range_miles)
    while not nearby_stations:
        range_miles *= 0.9
        nearby_stations = find_nearby_charging_stations(user_location, charging_stations, range_miles)
        if range_miles < 0.1:
            print("Error: No valid charging stations found within the specified range.")
            return

    # Calculate the distances between the user location and the nearby charging stations
    distances = calculate_distances([user_location] + nearby_stations)

    # Analyze the remainders of each EV model at each charging station
    remainders = analyze_remainders(distances, ev_models)

    # Optimize the charging station locations
    optimized_distances = optimize_charging_stations(distances, ev_models)

    # Evaluate the performance of the optimization algorithm
    performance = evaluate_performance(remainders, optimized_distances, ev_models)
    print(performance)

    # Visualize the results
    map_visualization = visualize_charging_stations([user_location] + nearby_stations, optimized_distances)
    map_visualization.save("charging_stations_map.html")